**Automatic ML - DS_C8_SC1_T5**

---


In [42]:
import pandas as pd

1. Leer en un dataframe, `apps`, el contenido del archivo GooglePlayStore.csv, que almacena información de las aplicaciones móviles para los dispositivos con sistema operativo Android. 

In [4]:
apps = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TLG/AutoML/DS_C8_SC1_T5_Pruebate/GooglePlayStore.csv")
apps

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10835,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10836,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1,4.1 and up
10837,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1,2.2 and up
10838,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


2.	Borra todos los registros que contengan valores faltantes y las columnas que no aportan a la predicción (`App`, `Last Updated`, `Current Ver`). No olvides codificar con *hot-encoding* las columnas categóricas previo al entrenamiento (recuerda que TPOT ocupa sólo valores numéricos) 

In [5]:
apps.dropna(inplace=True)
apps.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [6]:
apps.drop(['App','Last Updated','Current Ver'],axis=1,inplace=True)

In [7]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = apps.select_dtypes(include='object').columns.to_list()
apps = apps.apply(lambda x: LabelEncoder().fit_transform(x) if (x.name in categorical_cols) else x)

3.	Divide el dataframe en conjuntos de entrenamiento y prueba (train y test), considerando una distribución 70:30 e indica que la variable de respuesta o salida será *Rating*.

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(apps, train_size=0.70, random_state=1)

In [10]:
y = 'Rating'
X = list(apps.columns)
X.remove(y)

# **TPOT**

4.	Configura el AutoML considerando: 5 generaciones, 20  modelos a retener en cada generación, 10 pliegues para la validación cruzada, 3 minutos máximos para la generación de modelos y `r2` como métrica  para evaluar la calidad de un flujo. Ejecuta el entrenamiento.  

In [35]:
!pip install tpot
from tpot import TPOTRegressor

In [166]:
tpot = TPOTRegressor(generations=5, population_size=20, cv=10, max_time_mins=3, scoring='r2', random_state=1, verbosity=2)
tpot.fit(train[X], train[y])

Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


3.24 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestRegressor(XGBRegressor(input_matrix, learning_rate=0.001, max_depth=1, min_child_weight=9, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.55, verbosity=0), bootstrap=False, max_features=0.05, min_samples_leaf=5, min_samples_split=6, n_estimators=100)


TPOTRegressor(cv=10, generations=5, max_time_mins=3, population_size=20,
              random_state=1, scoring='r2', verbosity=2)

5.	Imprime el mejor modelo y evalúalo utilizando el el conjunto de prueba.

In [167]:
tpot.fitted_pipeline_

Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=XGBRegressor(base_score=0.5,
                                                          booster='gbtree',
                                                          colsample_bylevel=1,
                                                          colsample_bynode=1,
                                                          colsample_bytree=1,
                                                          enable_categorical=False,
                                                          gamma=0, gpu_id=-1,
                                                          importance_type=None,
                                                          interaction_constraints='',
                                                          learning_rate=0.001,
                                                          max_delta_step=0,
                                                          max_depth=1,
                       

In [168]:
tpot.score(test[X], test[y])

0.16159200625349712

# **PyCaret**

6. Configura el AutoML considerando: 10 pliegues para la validación cruzada y nada de preprocesamiento. Ejecuta la comparación de los modelos, limitando el tiempo a 3 minutos y usando `r2` como métrica para ordenar la tabla de puntuación. 

In [1]:
!pip uninstall scikit-learn -y
!pip uninstall pycaret -y
!pip install -U scikit-learn==0.23.2 imbalanced-learn==0.7.0 pycaret==2.2.1 
!pip install pyyaml==5.4.1

Found existing installation: scikit-learn 0.23.2
Uninstalling scikit-learn-0.23.2:
  Successfully uninstalled scikit-learn-0.23.2
Found existing installation: pycaret 2.2.1
Uninstalling pycaret-2.2.1:
  Successfully uninstalled pycaret-2.2.1
  Using cached scikit_learn-0.23.2-cp37-cp37m-manylinux1_x86_64.whl (6.8 MB)
  Using cached pycaret-2.2.1-py3-none-any.whl (248 kB)


In [3]:
import pycaret
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [12]:
pyc = setup(data=train, target='Rating', test_data=test, fold=10, preprocess=False, session_id=1)
leader = compare_models(budget_time=1.5, sort='r2')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.3285,2.397000e-01,0.4888,1.164000e-01,0.1093,0.0930,2.822
rf,Random Forest Regressor,0.3220,2.408000e-01,0.4898,1.138000e-01,0.1091,0.0911,1.993
gbr,Gradient Boosting Regressor,0.3391,2.443000e-01,0.4937,9.900000e-02,0.1104,0.0962,0.491
et,Extra Trees Regressor,0.3164,2.454000e-01,0.4943,9.650000e-02,0.1101,0.0898,1.300
br,Bayesian Ridge,0.3611,2.673000e-01,0.5164,1.480000e-02,0.1148,0.1025,0.024
lr,Linear Regression,0.3611,2.675000e-01,0.5166,1.400000e-02,0.1148,0.1025,0.377
ridge,Ridge Regression,0.3611,2.675000e-01,0.5165,1.400000e-02,0.1148,0.1025,0.025
lar,Least Angle Regression,0.3611,2.675000e-01,0.5166,1.400000e-02,0.1148,0.1025,0.027
en,Elastic Net,0.3623,2.686000e-01,0.5176,1.000000e-02,0.1150,0.1028,0.027
lasso,Lasso Regression,0.3624,2.690000e-01,0.5180,8.700000e-03,0.1151,0.1029,0.025


7. Selecciona el algoritmo que produce el mejor resultado en el conjunto de validación, para crear y optimizar el mejor modelo. 

In [13]:
leader = create_model(leader)
tuned_leader = tune_model(leader, optimize='r2') 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.3142,0.2190,0.4679,0.1016,0.1048,0.0879
1,0.3149,0.2001,0.4474,0.1932,0.0989,0.0862
2,0.3404,0.2431,0.4930,0.1245,0.1088,0.0956
3,0.3281,0.2428,0.4927,0.1216,0.1119,0.0954
4,0.3519,0.2688,0.5185,0.1365,0.1197,0.1046
5,0.3508,0.2622,0.5121,0.1188,0.1174,0.1030
6,0.3265,0.2247,0.4740,0.1457,0.1052,0.0914
7,0.3561,0.2620,0.5119,0.0914,0.1139,0.1017
8,0.3064,0.1867,0.4321,0.1329,0.0916,0.0804
9,0.3349,0.2502,0.5002,0.1439,0.1131,0.0967


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.3161,0.2114,0.4598,0.1575,0.1015,0.0877


,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Android Ver,Rating,Label
0,6.0,552441.0,102.0,6,0,72.0,1,34.0,16.0,4.4,4.400341
1,14.0,30630.0,38.0,2,0,72.0,1,86.0,16.0,4.2,4.289193
2,11.0,2683.0,87.0,5,0,72.0,3,94.0,12.0,4.6,4.157262
3,14.0,2464.0,130.0,14,0,72.0,1,109.0,16.0,4.6,4.196564
4,12.0,19.0,81.0,11,0,72.0,1,58.0,9.0,3.4,4.021796
...,...,...,...,...,...,...,...,...,...,...,...
2803,6.0,4604483.0,412.0,3,0,72.0,1,34.0,30.0,4.3,4.235129
2804,11.0,4449910.0,272.0,9,0,72.0,2,3.0,16.0,4.5,4.436466
2805,3.0,57400.0,24.0,17,0,72.0,1,18.0,12.0,4.8,4.382588
2806,25.0,253.0,327.0,5,0,72.0,1,80.0,16.0,4.4,4.065234


8. Evalúa el modelo optimizado en el conjunto de prueba. ¿Qué plataforma obtiene mejores resultados? 

In [14]:
predict_model(tuned_leader)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.3161,0.2114,0.4598,0.1575,0.1015,0.0877


,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Android Ver,Rating,Label
0,6.0,552441.0,102.0,6,0,72.0,1,34.0,16.0,4.4,4.400341
1,14.0,30630.0,38.0,2,0,72.0,1,86.0,16.0,4.2,4.289193
2,11.0,2683.0,87.0,5,0,72.0,3,94.0,12.0,4.6,4.157262
3,14.0,2464.0,130.0,14,0,72.0,1,109.0,16.0,4.6,4.196564
4,12.0,19.0,81.0,11,0,72.0,1,58.0,9.0,3.4,4.021796
...,...,...,...,...,...,...,...,...,...,...,...
2803,6.0,4604483.0,412.0,3,0,72.0,1,34.0,30.0,4.3,4.235129
2804,11.0,4449910.0,272.0,9,0,72.0,2,3.0,16.0,4.5,4.436466
2805,3.0,57400.0,24.0,17,0,72.0,1,18.0,12.0,4.8,4.382588
2806,25.0,253.0,327.0,5,0,72.0,1,80.0,16.0,4.4,4.065234
